In [1]:
import tensorflow as tf
import itertools

In [2]:
n = 3
d = 2*n
testMatrix = tf.reshape( tf.range(d*d*2), (2,d,d) )
print(testMatrix)

tf.Tensor(
[[[ 0  1  2  3  4  5]
  [ 6  7  8  9 10 11]
  [12 13 14 15 16 17]
  [18 19 20 21 22 23]
  [24 25 26 27 28 29]
  [30 31 32 33 34 35]]

 [[36 37 38 39 40 41]
  [42 43 44 45 46 47]
  [48 49 50 51 52 53]
  [54 55 56 57 58 59]
  [60 61 62 63 64 65]
  [66 67 68 69 70 71]]], shape=(2, 6, 6), dtype=int32)


In [3]:
rot = tf.transpose( tf.tensordot(tf.eye(d,d,dtype=tf.int32), tf.eye(d,d,dtype=tf.int32),0), [0,2,1,3])
rot_sub = tf.transpose( tf.tensordot(tf.eye(n,n,dtype=tf.int32), tf.eye(n,n,dtype=tf.int32),0), [0,2,1,3])
rot_cw = tf.reverse(tf.transpose(rot_sub, [1,0,2,3]), [0])
rot_cc = tf.reverse(tf.transpose(rot_sub, [1,0,2,3]), [1])
rotations = {}
s = [slice(0,n),slice(n,2*n)]
i = [0,1]
for perm in itertools.product(i, repeat=2):
    sectorName = f"{perm[0]}{perm[1]}"
    sliceArr = [s[perm[0]],s[perm[1]],s[perm[0]],s[perm[1]]]
    x = tf.Variable(rot)
    x = x.__getitem__(sliceArr).assign(rot_cw)
    rotations[f"{sectorName}cw"] = x
    x = tf.Variable(rot)
    x = x.__getitem__(sliceArr).assign(rot_cc)
    rotations[f"{sectorName}cc"] = x

In [4]:
for name, rot in rotations.items():
    res = tf.tensordot(testMatrix, rot, (2))
    print(name)
    print(res)

00cw
tf.Tensor(
[[[12  6  0  3  4  5]
  [13  7  1  9 10 11]
  [14  8  2 15 16 17]
  [18 19 20 21 22 23]
  [24 25 26 27 28 29]
  [30 31 32 33 34 35]]

 [[48 42 36 39 40 41]
  [49 43 37 45 46 47]
  [50 44 38 51 52 53]
  [54 55 56 57 58 59]
  [60 61 62 63 64 65]
  [66 67 68 69 70 71]]], shape=(2, 6, 6), dtype=int32)
00cc
tf.Tensor(
[[[ 2  8 14  3  4  5]
  [ 1  7 13  9 10 11]
  [ 0  6 12 15 16 17]
  [18 19 20 21 22 23]
  [24 25 26 27 28 29]
  [30 31 32 33 34 35]]

 [[38 44 50 39 40 41]
  [37 43 49 45 46 47]
  [36 42 48 51 52 53]
  [54 55 56 57 58 59]
  [60 61 62 63 64 65]
  [66 67 68 69 70 71]]], shape=(2, 6, 6), dtype=int32)
01cw
tf.Tensor(
[[[ 0  1  2 15  9  3]
  [ 6  7  8 16 10  4]
  [12 13 14 17 11  5]
  [18 19 20 21 22 23]
  [24 25 26 27 28 29]
  [30 31 32 33 34 35]]

 [[36 37 38 51 45 39]
  [42 43 44 52 46 40]
  [48 49 50 53 47 41]
  [54 55 56 57 58 59]
  [60 61 62 63 64 65]
  [66 67 68 69 70 71]]], shape=(2, 6, 6), dtype=int32)
01cc
tf.Tensor(
[[[ 0  1  2  5 11 17]
  [ 6  7  8  4 10

In [5]:
win_mapping_tensors = []
win_n = 5
for i, j in itertools.product(range(d),range(d-win_n+1)):
    spaces_map = tf.Variable(tf.zeros((d,d),dtype=tf.int32))
    spaces_map = spaces_map[i, j:win_n+j].assign(tf.ones((win_n),dtype=tf.int32))
    win_mapping_tensors.append(spaces_map)
    spaces_map = tf.Variable(tf.zeros((d,d),dtype=tf.int32))
    spaces_map = spaces_map[j:win_n+j, i].assign(tf.ones((win_n),dtype=tf.int32))
    win_mapping_tensors.append(spaces_map)
for i, j in itertools.product(range(d-win_n+1),repeat=2):
    spaces_map = tf.Variable(tf.zeros((d,d),dtype=tf.int32))
    spaces_map = spaces_map[i:win_n+i, j:win_n+j].assign(tf.eye(win_n,dtype=tf.int32))
    win_mapping_tensors.append(spaces_map)
    spaces_map = tf.Variable(tf.zeros((d,d),dtype=tf.int32))
    spaces_map = spaces_map[i:win_n+i, j:win_n+j].assign(tf.reverse(tf.eye(win_n,dtype=tf.int32),[0]))
    win_mapping_tensors.append(spaces_map)
Win = tf.stack(win_mapping_tensors, axis=2)

In [6]:
res = tf.reduce_max( tf.tensordot(testMatrix%2, Win, 2), [1] )
print(res)

tf.Tensor([5 5], shape=(2,), dtype=int32)


In [11]:
a = tf.equal(res, 5)
print(a)
if a[0]:
    print("test")

tf.Tensor([ True  True], shape=(2,), dtype=bool)
test
